In [1]:
import requests
import pandas as pd
from typing import Dict, List
from dataclasses import dataclass
from datetime import datetime
from web3 import Web3, HTTPProvider
import numpy as np
import matplotlib.pyplot as plt


@dataclass
class TokenInfo:
    symbol: str
    address: str

@dataclass
class MarketInfo:
    address: str
    factory_address: str
    llamma: str
    rate: float
    total_debt: float
    n_loans: int
    debt_ceiling: float
    borrowable: float
    pending_fees: float
    collected_fees: float
    collateral_amount: float
    collateral_amount_usd: float
    stablecoin_amount: float
    collateral_token: TokenInfo
    stablecoin_token: TokenInfo



In [2]:
def fetch_curve_markets(chain: str = "ethereum", page: int = 1, per_page: int = 10) -> Dict:
    """
    Fetch crvUSD market data from Curve API
    """
    url = f"https://prices.curve.fi/v1/crvusd/markets/{chain}"
    params = {
        "fetch_on_chain": "false",
        "page": page,
        "per_page": per_page
    }
    
    response = requests.get(url, params=params)
    response.raise_for_status()  # Raise exception for bad status codes
    return response.json()

def parse_market_data(market_data: Dict) -> MarketInfo:
    """
    Parse raw market data into MarketInfo object
    """
    
    w3 = Web3()
    
    return MarketInfo(
        address=w3.to_checksum_address(market_data["address"]),
        factory_address=w3.to_checksum_address(market_data["factory_address"]),
        llamma=w3.to_checksum_address(market_data["llamma"]),
        rate=market_data["rate"],
        total_debt=market_data["total_debt"],
        n_loans=market_data["n_loans"],
        debt_ceiling=market_data["debt_ceiling"],
        borrowable=market_data["borrowable"],
        pending_fees=market_data["pending_fees"],
        collected_fees=market_data["collected_fees"],
        collateral_amount=market_data["collateral_amount"],
        collateral_amount_usd=market_data["collateral_amount_usd"],
        stablecoin_amount=market_data["stablecoin_amount"],
        collateral_token=TokenInfo(
            symbol=market_data["collateral_token"]["symbol"],
            address=w3.to_checksum_address(market_data["collateral_token"]["address"])
        ),
        stablecoin_token=TokenInfo(
            symbol=market_data["stablecoin_token"]["symbol"],
            address=w3.to_checksum_address(market_data["stablecoin_token"]["address"])
        )
    )

def get_markets_df() -> pd.DataFrame:
    """
    Fetch market data and return as a pandas DataFrame
    """
    raw_data = fetch_curve_markets()
    markets = [parse_market_data(market) for market in raw_data["data"]]
    
    # Convert to dictionary format suitable for DataFrame
    markets_dict = {
        "address": [],
        "collateral_token": [],
        "rate": [],
        "total_debt": [],
        "n_loans": [],
        "borrowable": [],
        "collateral_amount": [],
        "collateral_amount_usd": [],
        "pending_fees": [],
        "collected_fees": []
    }
    
    for market in markets:
        markets_dict["address"].append(market.address)
        markets_dict["collateral_token"].append(market.collateral_token.symbol)
        markets_dict["rate"].append(market.rate)
        markets_dict["total_debt"].append(market.total_debt)
        markets_dict["n_loans"].append(market.n_loans)
        markets_dict["borrowable"].append(market.borrowable)
        markets_dict["collateral_amount"].append(market.collateral_amount)
        markets_dict["collateral_amount_usd"].append(market.collateral_amount_usd)
        markets_dict["pending_fees"].append(market.pending_fees)
        markets_dict["collected_fees"].append(market.collected_fees)
    
    return pd.DataFrame(markets_dict)

def get_markets_map() -> Dict[str, MarketInfo]:
    """
    Fetch market data and return as a dictionary mapping addresses to MarketInfo objects
    """
    raw_data = fetch_curve_markets()
    markets_map = {}
    
    for market in raw_data["data"]:
        if market["borrowable"] != 0:
            markets_map[market["collateral_token"]["symbol"]] = parse_market_data(market)

    return markets_map


In [3]:
markets = get_markets_map()
markets

{'wstETH': MarketInfo(address='0x100dAa78fC509Db39Ef7D04DE0c1ABD299f4C6CE', factory_address='0xC9332fdCB1C491Dcc683bAe86Fe3cb70360738BC', llamma='0x37417B2238AA52D0DD2D6252d989E728e8f706e4', rate=0.032969654385967884, total_debt=12508840.186299857, n_loans=85, debt_ceiling=None, borrowable=137494437.9547297, pending_fees=3278.1410295642577, collected_fees=6949295.01963203, collateral_amount=5251.009616338953, collateral_amount_usd=19715084.7876002, stablecoin_amount=20874.032413869943, collateral_token=TokenInfo(symbol='wstETH', address='0x7f39C581F595B53c5cb19bD0b3f8dA6c935E2Ca0'), stablecoin_token=TokenInfo(symbol='crvUSD', address='0xf939E0A03FB07F59A73314E73794Be0E57ac1b4E')),
 'WBTC': MarketInfo(address='0x4e59541306910aD6dC1daC0AC9dFB29bD9F15c67', factory_address='0xC9332fdCB1C491Dcc683bAe86Fe3cb70360738BC', llamma='0xE0438Eb3703bF871E31Ce639bd351109c88666ea', rate=0.02044131503970914, total_debt=19948803.894962903, n_loans=157, debt_ceiling=None, borrowable=180053406.29274133, p

In [4]:
def fetch_liquidation_overview(market_address: str, chain: str = "ethereum") -> Dict:
    """
    Fetch liquidation overview data for a specific crvUSD market
    
    Parameters:
    -----------
    market_address : str
        The market address to fetch data for
    chain : str
        The blockchain network (default: "ethereum")
        
    Returns:
    --------
    Dict containing liquidation metrics including:
        - soft_liquidation_users
        - median_health
        - average_health
        - collat_ratio
        - bad_debt
        - liquidatable_positions
        - liquidatable metrics (debt, stablecoin, collateral) in both raw and USD terms
    """
    # Ensure we're using checksum address
    w3 = Web3()
    market_address = w3.to_checksum_address(market_address)
    
    url = f"https://prices.curve.fi/v1/crvusd/liquidations/{chain}/{market_address}/overview"
    params = {
        "fetch_on_chain": "false"
    }
    
    response = requests.get(url, params=params)
    response.raise_for_status()
    
    return response.json()



---

In [5]:
def score_debt_ceiling(recommended_debt_ceiling: float,
                       current_debt_ceiling: float,
                       current_debt: float) -> float:
    if current_debt_ceiling <= recommended_debt_ceiling:
        return 1.0
    elif current_debt <= recommended_debt_ceiling:
        # score between 0.5 and 1
        return 0.5 + 0.5 * ((recommended_debt_ceiling - current_debt) / recommended_debt_ceiling)
    else:
        return 0.0

In [6]:
recommended_ceilings = {
    "wstETH": 110000000,
    "WBTC": 80000000,
    "WETH": 220000000,
    "sfrxETH": 15000000,
    "tBTC": 20000000
}



---

In [8]:
print("\nDEBT CEILING SCORING OVERVIEW")
print("=" * 95)
print(f"{'Market':8} | {'Total Debt':>12} | {'Borrowable':>12} | {'Current Cap':>12} | {'Recommended':>12} | {'Score':>8}")
print("-" * 95)

for market, info in markets.items():
    total_debt = info.total_debt
    borrowable = info.borrowable
    debt_ceiling = total_debt + borrowable
    recommended_debt_ceiling = recommended_ceilings[market]
    score = score_debt_ceiling(recommended_debt_ceiling, debt_ceiling, total_debt)
    
    # Format numbers for better readability
    total_debt_fmt = f"${total_debt/1e6:.2f}M"
    borrowable_fmt = f"${borrowable/1e6:.2f}M"
    ceiling_fmt = f"${debt_ceiling/1e6:.2f}M"
    recommended_fmt = f"${recommended_debt_ceiling/1e6:.2f}M"
    
    print(f"{market:8} | {total_debt_fmt:>12} | {borrowable_fmt:>12} | {ceiling_fmt:>12} | {recommended_fmt:>12} | {score*100:>7.2f}%")

print("=" * 95)


DEBT CEILING SCORING OVERVIEW
Market   |   Total Debt |   Borrowable |  Current Cap |  Recommended |    Score
-----------------------------------------------------------------------------------------------
wstETH   |      $12.51M |     $137.49M |     $150.00M |     $110.00M |   94.31%
WBTC     |      $19.95M |     $180.05M |     $200.00M |      $80.00M |   87.53%
WETH     |      $32.55M |     $167.45M |     $200.00M |     $220.00M |  100.00%
sfrxETH  |       $0.42M |      $49.58M |      $50.00M |      $15.00M |   98.59%
tBTC     |       $5.49M |      $44.51M |      $50.00M |      $20.00M |   86.27%
